In [ ]:
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM,Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

In [ ]:
#creates dataset with keratocyte data
#creates dataset to use for training/testing - updated scaling
#first 50*train_prop cells of each EF used for training and remaing are testing
#thus, this function not compatible with shuffled k-fold CV
def create_dataset(df, lookback=20, in_cols=['volt', 'cum_dir'], out_cols='cum_dir', efs=[0,15,30,50,75,100,200], tracks=(1,51)):
    trainX, trainY = [], [] #lists of inputs/outputs
    efs = df['ef'].unique()
    for ef in efs:
        maxtrack = max(df.loc[df['ef']==ef, 'track'])
        for track in range(tracks[0], maxtrack+1):
            cell = df.loc[(df["ef"] == ef) & (df["track"] == track)] #all rows of data pertaining to this cell
            cell = cell[in_cols] #reduce it to our columns of interest
            for i in range(len(cell)-lookback-1):
                trainX.append(cell[i:i+lookback])
            cell = cell[out_cols]
            for i in range(len(cell)-lookback-1):
                trainY.append(cell[i+lookback:i+lookback+1])

    trainX = np.array(list(map(lambda x: x.to_numpy(), trainX)))
    trainY = np.array(list(map(lambda x: x.to_numpy(), trainY)))
    return np.array(trainX), np.array(trainY)

In [ ]:
keratocyte = pd.read_csv('data/keratocyte.csv')
print('Max slice: {}, Max track: {}'.format(max(keratocyte['slice']), max(keratocyte['track'])))
print(max(keratocyte['volt']))
keratocyte.head()

In [ ]:
# Create train, validation, and test sets for keratocyte
trainX, trainY = create_dataset(keratocyte.loc[keratocyte['track']<3])
valX, valY = create_dataset(keratocyte.loc[(keratocyte['track']>=3)&(keratocyte['track']<10)])
testX, testY = create_dataset(keratocyte.loc[keratocyte['track']>=10])
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)
print(testX.shape)
print(testY.shape)

models = [] #list of models 
for i in range(50):
    #build the model
    print('Training model number {}'.format(i))
    model = Sequential()
    model.add(LSTM(80, input_shape=(20, 2)))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=1, verbose=1)
    models.append(model)
        
    model.save("models/keratocyte_lstm/model{}.h5".format(i))
    print("Saved model {} to disk".format(i))

    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))
        
    #make predictions on each instance and add to list of prediction vectors
    train_pred = model.predict(trainX)
    val_pred = model.predict(valX)
    test_pred = model.predict(testX)

In [ ]:
keratocyte = pd.read_csv('data/keratocyte.csv')

trainX, trainY = create_dataset(keratocyte.loc[keratocyte['track']<3])
valX, valY = create_dataset(keratocyte.loc[(keratocyte['track']>=3)&(keratocyte['track']<10)])
testX, testY = create_dataset(keratocyte.loc[keratocyte['track']>=10])

# print results to file
i = 0
for model in models:
    print('Writing results for model {}'.format(i))
    efs = keratocyte['ef'].unique()
    for ef in efs:
        maxtrack = max(keratocyte.loc[keratocyte['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratocyte.loc[(keratocyte['ef']==ef) & (keratocyte['track']==track)]
            if len(cell)==0:
                continue
            if ef==1000 and track==7:
                continue
            maxslice = max(keratocyte.loc[(keratocyte['ef']==ef)&(keratocyte['track']==track), 'slice'])
            minslice = min(keratocyte.loc[(keratocyte['ef']==ef)&(keratocyte['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratocyte.loc[(keratocyte['ef']==ef) & (keratocyte['track']==track) & (keratocyte['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
                
    keratocyte['pred_error{}'.format(i)] = keratocyte['pred_dir{}'.format(i)] - keratocyte['cum_dir']
    i+=1
keratocyte.to_csv('data/keratocyte_predictions.csv', index=False)
keratocyte.head(40)

In [ ]:
keratocyte = pd.read_csv('data/keratocyte.csv')

i=0
for filename in os.listdir('models/cncc_lstm'):
    print('Loading models/cncc_lstm/{}'.format(filename))
    
    # now do transfer learning using cncc
    model = load_model('models/cncc_lstm/{}'.format(filename))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=20, batch_size=1, verbose=1)
    model.save("models/keratocyte_lstm_transfer/model{}.h5".format(i))
    print("Saved model{}.h5 to disk".format(i))
    
    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))
    
    print('Writing results for model {}'.format(i))
    efs = keratocyte['ef'].unique()
    for ef in efs:
        maxtrack = max(keratocyte.loc[keratocyte['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratocyte.loc[(keratocyte['ef']==ef) & (keratocyte['track']==track)]
            if len(cell)==0:
                continue
            if ef==1000 and track==7:
                continue
            maxslice = max(keratocyte.loc[(keratocyte['ef']==ef)&(keratocyte['track']==track), 'slice'])
            minslice = min(keratocyte.loc[(keratocyte['ef']==ef)&(keratocyte['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratocyte.loc[(keratocyte['ef']==ef) & (keratocyte['track']==track) & (keratocyte['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
                
    keratocyte['pred_error{}'.format(i)] = keratocyte['pred_dir{}'.format(i)] - keratocyte['cum_dir']
    i+=1
keratocyte.to_csv('data/keratocyte_transfer_predictions.csv', index=False)
keratocyte.head(40)

In [ ]:
keratino1 = pd.read_csv('data/NHK0001.csv')
keratino1['slice'] = keratino1['slice']+1
keratino1['volt'] = 0.1
keratino1['ef'] = 100
print('Max slice: {}, Max track: {}'.format(max(keratino1['slice']), max(keratino1['track'])))
keratino1.head()

In [ ]:
# Create train, validation, and test sets for NHK0001 keratinocytes
trainX, trainY = create_dataset(keratino1.loc[keratino1['track']<3])
valX, valY = create_dataset(keratino1.loc[(keratino1['track']>=3)&(keratino1['track']<5)])
testX, testY = create_dataset(keratino1.loc[keratino1['track']>=5])
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)
print(testX.shape)
print(testY.shape)

models = [] #list of models 
for i in range(50):
    # Build and train model
    model = Sequential()
    model.add(LSTM(80, input_shape=(20, 2)))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=1, verbose=1)
    models.append(model)
    
    model.save("models/NHK0001_lstm/model{}.h5".format(i))
    print("Saved model {} to disk".format(i))

    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))

In [ ]:
keratino1 = pd.read_csv('data/NHK0001.csv')
keratino1['slice'] = keratino1['slice']+1
keratino1['volt'] = 0.1
keratino1['ef'] = 100

trainX, trainY = create_dataset(keratino1.loc[keratino1['track']<3])
valX, valY = create_dataset(keratino1.loc[(keratino1['track']>=3)&(keratino1['track']<5)])
testX, testY = create_dataset(keratino1.loc[keratino1['track']>=5])

# print results to file
i = 0
for model in models:
    print('Writing results for model {}'.format(i))
    efs = keratino1['ef'].unique()
    for ef in efs:
        maxtrack = max(keratino1.loc[keratino1['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratino1.loc[(keratino1['ef']==ef) & (keratino1['track']==track)]
            if len(cell)==0:
                continue
            if ef==1000 and track==7:
                continue
            maxslice = max(keratino1.loc[(keratino1['ef']==ef)&(keratino1['track']==track), 'slice'])
            minslice = min(keratino1.loc[(keratino1['ef']==ef)&(keratino1['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratino1.loc[(keratino1['ef']==ef) & (keratino1['track']==track) & (keratino1['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
                
    keratino1['pred_error{}'.format(i)] = keratino1['pred_dir{}'.format(i)] - keratino1['cum_dir']
    i+=1
keratino1.to_csv('data/NHK0001_predictions.csv', index=False)
keratino1.head(40)

In [ ]:
keratino1 = pd.read_csv('data/NHK0001.csv')
keratino1['slice'] = keratino1['slice']+1
keratino1['volt'] = 0.1
keratino1['ef'] = 100

i=0
for filename in os.listdir('models/cncc_lstm'):
    print('Loading models/cncc_lstm/{}'.format(filename))
    
    # now do transfer learning using cncc
    model = load_model('models/cncc_lstm/{}'.format(filename))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=20, batch_size=1, verbose=1)
    model.save("models/NHK0001_lstm_transfer/model{}.h5".format(i))
    print("Saved model{}.h5 to disk".format(i))
    
    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))
    
    print('Writing results for model {}'.format(i))
    efs = keratino1['ef'].unique()
    for ef in efs:
        maxtrack = max(keratino1.loc[keratino1['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratino1.loc[(keratino1['ef']==ef) & (keratino1['track']==track)]
            if len(cell)==0:
                continue
            if ef==1000 and track==7:
                continue
            maxslice = max(keratino1.loc[(keratino1['ef']==ef)&(keratino1['track']==track), 'slice'])
            minslice = min(keratino1.loc[(keratino1['ef']==ef)&(keratino1['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratino1.loc[(keratino1['ef']==ef) & (keratino1['track']==track) & (keratino1['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
    keratino1['pred_error{}'.format(i)] = keratino1['pred_dir{}'.format(i)] - keratino1['cum_dir']
    i+=1
keratocyte.to_csv('data/NHK0001_transfer_predictions.csv', index=False)
keratocyte.head(40)

In [ ]:
keratino2 = pd.read_csv('data/NHK0002.csv')
keratino2['slice'] = keratino2['slice']+1
keratino2['volt'] = 0.1
keratino2['ef'] = 100
print('Max slice: {}, Max track: {}'.format(max(keratino2['slice']), max(keratino2['track'])))
keratino2.head()

In [ ]:
# Create train, validation, and test sets for NHK0002 keratinocytes
trainX, trainY = create_dataset(keratino2.loc[keratino2['track']<3])
valX, valY = create_dataset(keratino2.loc[(keratino2['track']>=3)&(keratino2['track']<5)])
testX, testY = create_dataset(keratino2.loc[keratino2['track']>=5])
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)
print(testX.shape)
print(testY.shape)

models = [] #list of models 
for i in range(50):
    # Build and train model
    model = Sequential()
    model.add(LSTM(80, input_shape=(20, 2)))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=1, verbose=1)
    models.append(model)
    
    model.save("models/NHK0002_lstm/model{}.h5".format(i))
    print("Saved model {} to disk".format(i))

    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))

In [ ]:
keratino2 = pd.read_csv('data/NHK0002.csv')
keratino2['slice'] = keratino2['slice']+1
keratino2['volt'] = 0.1
keratino2['ef'] = 100

trainX, trainY = create_dataset(keratino2.loc[keratino2['track']<3])
valX, valY = create_dataset(keratino2.loc[(keratino2['track']>=3)&(keratino2['track']<5)])
testX, testY = create_dataset(keratino2.loc[keratino2['track']>=5])

# print results to file
i = 0
for model in models:
    print('Writing results for model {}'.format(i))
    efs = keratino2['ef'].unique()
    for ef in efs:
        maxtrack = max(keratino2.loc[keratino2['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratino2.loc[(keratino2['ef']==ef) & (keratino2['track']==track)]
            if len(cell)==0:
                continue
            maxslice = max(keratino2.loc[(keratino2['ef']==ef)&(keratino2['track']==track), 'slice'])
            minslice = min(keratino2.loc[(keratino2['ef']==ef)&(keratino2['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratino2.loc[(keratino2['ef']==ef) & (keratino2['track']==track) & (keratino2['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
    keratino2['pred_error{}'.format(i)] = keratino2['pred_dir{}'.format(i)] - keratino2['cum_dir']
    i+=1
keratino2.to_csv('data/NHK0002_predictions.csv', index=False)
keratino2.head(40)

In [ ]:
keratino2 = pd.read_csv('data/NHK0002.csv')
keratino2['slice'] = keratino2['slice']+1
keratino2['volt'] = 0.1
keratino2['ef'] = 100

i=0
for filename in os.listdir('models/cncc_lstm'):
    print('Loading models/cncc_lstm/{}'.format(filename))
    
    # now do transfer learning using cncc
    model = load_model('models/cncc_lstm/{}'.format(filename))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=20, batch_size=1, verbose=1)
    model.save("models/NHK0002_lstm_transfer/model{}.h5".format(i))
    print("Saved model{}.h5 to disk".format(i))
    
    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))
    
    print('Writing results for model {}'.format(i))
    efs = keratino2['ef'].unique()
    for ef in efs:
        maxtrack = max(keratino2.loc[keratino2['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratino2.loc[(keratino2['ef']==ef) & (keratino2['track']==track)]
            if len(cell)==0:
                continue
            maxslice = max(keratino2.loc[(keratino2['ef']==ef)&(keratino2['track']==track), 'slice'])
            minslice = min(keratino2.loc[(keratino2['ef']==ef)&(keratino2['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratino2.loc[(keratino2['ef']==ef) & (keratino2['track']==track) & (keratino2['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
    keratino2['pred_error{}'.format(i)] = keratino2['pred_dir{}'.format(i)] - keratino2['cum_dir']
    i+=1
    
keratino2.to_csv('data/NHK0002_transfer_predictions.csv', index=False)
keratino2.head(40)

In [ ]:
keratino3 = pd.read_csv('data/NHK0802-YL112208.csv')
keratino3['slice'] = keratino3['slice']+1
keratino3['volt'] = 0.1
keratino3['ef'] = 100
print('Max slice: {}, Max track: {}'.format(max(keratino3['slice']), max(keratino3['track'])))
keratino3.head()

In [ ]:
# Create train, validation, and test sets for NHK0802-YL112208 keratinocytes
trainX, trainY = create_dataset(keratino3.loc[keratino3['track']<3])
valX, valY = create_dataset(keratino3.loc[(keratino3['track']>=3)&(keratino3['track']<10)])
testX, testY = create_dataset(keratino3.loc[keratino3['track']>=10])
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)
print(testX.shape)
print(testY.shape)

models = [] #list of models 
for i in range(50):
    # Build and train model
    model = Sequential()
    model.add(LSTM(80, input_shape=(20, 2)))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=100, batch_size=1, verbose=1)
    models.append(model)
    
    model.save("models/NHK0802-YL112208_lstm/model{}.h5".format(i))
    print("Saved model {} to disk".format(i))

    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))

In [ ]:
keratino3 = pd.read_csv('data/NHK0802-YL112208.csv')
keratino3['slice'] = keratino3['slice']+1
keratino3['volt'] = 0.1
keratino3['ef'] = 100

trainX, trainY = create_dataset(keratino3.loc[keratino3['track']<3])
valX, valY = create_dataset(keratino3.loc[(keratino3['track']>=3)&(keratino3['track']<10)])
testX, testY = create_dataset(keratino3.loc[keratino3['track']>=10])

# print results to file
i = 0
for model in models:
    print('Writing results for model {}'.format(i))
    efs = keratino3['ef'].unique()
    for ef in efs:
        maxtrack = max(keratino3.loc[keratino3['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratino3.loc[(keratino3['ef']==ef) & (keratino3['track']==track)]
            if len(cell)==0:
                continue
            maxslice = max(keratino3.loc[(keratino3['ef']==ef)&(keratino3['track']==track), 'slice'])
            minslice = min(keratino3.loc[(keratino3['ef']==ef)&(keratino3['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratino3.loc[(keratino3['ef']==ef) & (keratino3['track']==track) & (keratino3['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
    keratino3['pred_error{}'.format(i)] = keratino3['pred_dir{}'.format(i)] - keratino3['cum_dir']
    i+=1
keratino3.to_csv('data/NHK0802-YL112208_predictions.csv', index=False)
keratino3.head(40)

In [ ]:
keratino3 = pd.read_csv('data/NHK0802-YL112208.csv')
keratino3['slice'] = keratino3['slice']+1
keratino3['volt'] = 0.1
keratino3['ef'] = 100
trainX, trainY = create_dataset(keratino3.loc[keratino3['track']<3])
valX, valY = create_dataset(keratino3.loc[(keratino3['track']>=3)&(keratino3['track']<10)])
testX, testY = create_dataset(keratino3.loc[keratino3['track']>=10])

i=0
for filename in os.listdir('models/cncc_lstm'):
    print('Loading models/cncc_lstm/{}'.format(filename))
    
    # now do transfer learning using cncc
    model = load_model('models/cncc_lstm/{}'.format(filename))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=20, batch_size=1, verbose=1)
    model.save("models/NHK0802-YL112208_lstm_transfer/model{}.h5".format(i))
    print("Saved model{}.h5 to disk".format(i))
    
    #Predict on training, validation, and test sets
    trainPredict = model.predict(trainX)
    valPredict = model.predict(valX)
    testPredict = model.predict(testX)

    #Calculate RMSEs
    trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
    print('Training RMSE: {}'.format(trainScore))
    valScore = math.sqrt(mean_squared_error(valY, valPredict))
    print('Validation RMSE: {}'.format(valScore))
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Testing RMSE: {}'.format(testScore))
    
    print('Writing results for model {}'.format(i))
    efs = keratino3['ef'].unique()
    for ef in efs:
        maxtrack = max(keratino3.loc[keratino3['ef']==ef, 'track'])
        for track in range(1, maxtrack+1):
            cell = keratino3.loc[(keratino3['ef']==ef) & (keratino3['track']==track)]
            if len(cell)==0:
                continue
            maxslice = max(keratino3.loc[(keratino3['ef']==ef)&(keratino3['track']==track), 'slice'])
            minslice = min(keratino3.loc[(keratino3['ef']==ef)&(keratino3['track']==track), 'slice'])
            for sl in range(minslice+20,maxslice-minslice+2):
                x = cell.loc[(cell['slice']>sl-21) & (cell['slice']<sl)]
                x = x[['volt','cum_dir']].to_numpy()
                x=x.reshape(1, 20, 2)
                keratino3.loc[(keratino3['ef']==ef) & (keratino3['track']==track) & (keratino3['slice']==sl), 'pred_dir{}'.format(i)] = model.predict(x)
    keratino3['pred_error{}'.format(i)] = keratino3['pred_dir{}'.format(i)] - keratino3['cum_dir']
    i+=1
keratino3.to_csv('data/NHK0802-YL112208_transfer_predictions.csv', index=False)
keratino3.head(40)